sign_change.py: Generate positive or negative average weight changes at each electrode

## Pseudocode:
    . Import
    . Input symptom, mode, perdur
    . Define functions
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import xlsxwriter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.io import loadmat
from scipy.signal import butter,filtfilt
from scipy.stats import ttest_1samp

## Input symptom, mode, and duration

In [2]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual']
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC'];
    
abv_neuroanat_list = ['front-pole', #FRONTAL LOBE
    'parstri',
    'parsop',
    'parsorb',
    'rost-midfront',
    'caud-midfront',
    'latorb-front',
    'sup-front',
    'medorb-front',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inf-par',   
    'sup-par',
    'supra-marg',
    'temp-pole', # TEMPORAL LOBE
    'mid-temp',
    'sup-temp',
    'inf-temp',
    'parahip',               
    'R-Hip',
    'L-Hip',
    'R-Amyg',
    'L-Amyg',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual']
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC']
# sx_list = ['lex',
#            'lhx',
#            'lnx',
#            'lfx',
#            'ltx',
#            'lmx',
#            'lud',
#            'lup',
#            'lld',
#            'llp',
#            'rex',
#            'rhx',
#            'rnx',
#            'rfx',
#            'rtx',
#            'rmx',
#            'rud',
#            'rup',
#            'rld',
#            'rlp',
#            'brx',
#            'fax',
#            'oax',
#            'qcx']
sx_list = ['lhx','lup','lud','rhx','rup','rud']
mx_list = ['2']
# mx_list = ['1','2','3']
perdur_input = 15

## Define Functions

In [3]:
#LINELENGTH MEAN DIFFERENCE BEFORE AND AFTER SYMPTOM ONSET 

def sem_w8s(LL,at_onset,before_onset,after_onset):
    row_num = np.shape(LL)[0]
    LL_meandiff = np.empty(row_num,)
    for row in range(0,row_num-1): #for each channel in LL   
         LL_meandiff[row] = np.mean(LL[row,int(at_onset):int(after_onset)]) - np.mean(LL[row,int(before_onset):int(at_onset)])
    return LL_meandiff

In [4]:
#LINELENGTH TRANSFORM

def ll_transform(llw,fs,d,bl_start,bl_stop):    
    if bl_start == 0:
        sample_bl_start = 0
    elif bl_start > 0:
        sample_bl_start = (fs*bl_start)[0] #0 because it's python not matlab
    sample_bl_end = (fs*bl_stop)[0]-1
    
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    
    col_len = np.shape(d)[1]-L
    
    LL = np.empty([np.shape(d)[0],col_len])
    
    LL[:] = np.NaN        
    
    for col_1 in range(0,col_len):
        LL[:,col_1] = np.sum(np.abs(np.diff(d[:,col_1:col_1+L])),1)


    for row_1 in range(0,np.shape(d)[0]):    
        LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
        LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])
                                              
        LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd
                    
    return LL


In [5]:
#XLSX --> PANDA DATAFRAME 

def open_xl(xl_name): 
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df

In [6]:
#FILTER PPEEG TO PREPARE FOR LINELENGTH TRANSFORM

def filt(d,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')
    
    filt_d = filtfilt(b,a,d_t,axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose()

    return filt_d


In [7]:
#LOAD MAT FILES 

def load_elecs(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anat = e_mat['anatomy'][:,3]
    
    em = e_mat['elecmatrix']
    
    em_len = np.shape(em)[0]
    anat_len = np.shape(anat)[0]

    #delete excess rows in elecmatrix and anatomy
    if np.shape(em)[0] < np.shape(anat)[0]:
        anat = np.delete(anat,[em_len,anat_len-1])        
    elif np.shape(em)[0] > np.shape(anat)[0]:
        em = np.delete(em,[anat_len,em_len-1]) 

    return anat,em

In [8]:
#LOAD OPSCEA PARAMETERS 

def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [9]:
#LOAD PATIENT AND SEIZURE NAMES AND LATERALITY

def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        direction_data = []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
            direction_data.append(sz_names_list[r][2])
    return pt_data, sz_data, direction_data, df_params

## For loop through patients in list matching seizure

In [10]:
# def neurosem(sx_input,mx_input,perdur_input,pt_data,sz_data,df_params,workbook):
opscea_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/'

avg_change_path = opscea_path+'OPSCEADATA/avg_change_folders/'

pt_data,sz_data,direction_data,df_params = get_ptsz(avg_change_path) #Ehsan

# create xlsx workbook 
sign_workbook = xlsxwriter.Workbook(opscea_path+'sign_change_LL.xlsx')
em_workbook = xlsxwriter.Workbook(opscea_path+'em_LL.xlsx')

#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan
    direction = direction_data[i]

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
            print(sz_name)
            # PARAMS
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw

            # ANAT 
            anat,em = load_elecs(pt_path)
            
            emnan = np.isnan(em)
            em[emnan] = 0;

            # SEMIOLOGY
            sz_path = pt_path + sz_data[i]
            os.chdir(sz_path)

            sz_mat = loadmat(sz_name +'.mat') #load frame speed and ppEEG
            fs = sz_mat['fs'].flatten()
            d = sz_mat['ppEEG'][0:np.shape(anat)[0],:]

            badch_mat = loadmat(sz_name + '_badch.mat')
            badch = badch_mat['bad_chs']
            badch = np.delete(badch,range(np.shape(anat)[0],np.shape(badch)[0]),0)
            
            for a_i in range(0,np.shape(anat)[0]):
                if np.size(anat[a_i]) > 0: 
                    if anat[a_i][0] not in neuroanat_list:
                        badch[a_i][0] = 1 #prepare for anatomy not in neuroanat list to be deleted in d and anat

            bad_logical = np.any(badch,1)
            d = d[~bad_logical,:]
            anat = anat[~bad_logical]
            em = em[~bad_logical,:]

            sx_count = 0
            for sx_input in sx_list:
                sx_count += 1
                for mx_input in mx_list:
                    print(sx_input,' ',mx_input)

#                     XLSX

                    sign_worksheet_name = sx_input+' '+mx_input
                    em_worksheet_name = sx_input+' '+mx_input

                    if i == 0:
                        # add 1 xlsx worksheet per symptom/mode
                        sign_worksheet = sign_workbook.add_worksheet(sign_worksheet_name) 
                        # sign_worksheet.write(0,0,sign_worksheet_name) #label symptom 
                        sign_worksheet.write(0, i, direction_data[i]) #write laterality of onset
                        sign_worksheet.write(1, i, sz_data[i]) #write patient seizure name
                        
                        em_worksheet = em_workbook.add_worksheet(em_worksheet_name) 
                        for each_col in range(0,np.shape(em)[1]):
                            if (each_col % 3) == 0:
                                em_worksheet.write(0, 3*i+each_col, direction_data[i]) #write laterality of onset
                                em_worksheet.write(1, 3*i+each_col, sz_data[i]) #write patient seizure name        
                        
                    elif i > 0:
                        sign_worksheet = sign_workbook.get_worksheet_by_name(sign_worksheet_name)
                        sign_worksheet.write(0, i, direction_data[i]) #write laterality of onset
                        sign_worksheet.write(1, i, sz_data[i]) #write patient seizure name        
                        
                        em_worksheet = em_workbook.get_worksheet_by_name(em_worksheet_name) 
                        for each_col in range(0,np.shape(em)[1]):
                            # em_worksheet.write(0, 3*i+each_col, direction_data[i]) #write laterality of onset
                            # em_worksheet.write(1, 3*i+each_col, sz_data[i]) #write patient seizure name        
                            if (each_col % 3) == 0:
                                em_worksheet.write(0, 3*i+each_col, direction_data[i]) #write laterality of onset
                                em_worksheet.write(1, 3*i+each_col, sz_data[i]) #write patient seizure name        


                    sx_vec = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]).values.tolist() #load semiology matrix .csv             #WANT TO JUST LOAD 1 COLUMN SX_INPUT            

                    if np.float64(mx_input) in sx_vec:
                        
                        filt_d = filt(d,fs) #Filter out < 1 Hz (and up to nyquist)

                        LL = ll_transform(llw,fs,filt_d,blstart,blstop) 

                        at_onset = np.round(((sx_vec.index(np.float64(mx_input)))/5) * fs)
                        before_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) - float(perdur_input)) * fs)
                        after_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) + float(perdur_input)) * fs)

                        LL_meandiff = sem_w8s(LL,at_onset,before_onset,after_onset)
                                                
                        for each_row in range(0,LL_meandiff.size):
                            sign_worksheet.write(each_row+2,i,LL_meandiff.item(each_row))
                            
                            for each_col in range(0,np.shape(em)[1]):
                                em_worksheet.write(each_row+2,3*i+each_col,em[each_row][each_col])
            
sign_workbook.close()
em_workbook.close()

EC91_03
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2
EC96_01
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2
EC107_01
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2
EC133_03
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2
EC166_01
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2
EC228_03
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2
EC229_02
lhx   2
lup   2
lud   2
rhx   2
rup   2
rud   2


In [11]:
# tstat, pval = ttest_1samp([-33.00863798812543, -7.4380449766326855],0)
# print(pval)
